In [1]:
import pandas as pd
from tensorflow import keras
import tensorflow as tf
import random
from sklearn.utils import shuffle
import scipy.stats
from zipfile import ZipFile 
import optuna
from sklearn.model_selection import train_test_split

2024-04-26 02:48:46.785266: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-26 02:48:47.418232: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
import time
from tensorflow.keras import layers, regularizers
from tensorflow.keras.utils import to_categorical

In [4]:
import os
import shutil
from scipy import io #for loadmat, matlab conversion
import numpy as np
import matplotlib.pyplot as plt # for plotting training curves
#from tensorflow import keras #added to save model
from tensorflow.keras import layers #format matches MNIST example
#imports for computing and displaying output metrics
import seaborn as sns
import pandas as pd
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, confusion_matrix, precision_recall_fscore_support

In [5]:
train_file = 'train_mitbih_new.csv'
test_file = 'test_mitbih_new.csv'
data_train = pd.read_csv(train_file, header=0)
data_test = pd.read_csv(test_file, header=0)
print(data_train.shape)
print(data_test.shape)

x_train = data_train.iloc[:, :-1].values
y_train = data_train.iloc[:, -1].values
x_test = data_test.iloc[:, :-1].values
y_test = data_test.iloc[:, -1].values

y_train = to_categorical(y_train, num_classes=5)
y_test = to_categorical(y_test, num_classes=5)
x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], 1)
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], 1)
x_train, x_validate, y_train, y_validate = train_test_split(x_train, y_train, test_size=0.25, stratify=y_train, shuffle=True)

(22330, 188)
(5586, 188)


In [11]:
def cnn_model_1(x_train, y_train):
    n_timesteps, n_features, n_outputs = x_train.shape[1], x_train.shape[2], y_train.shape[1]
    model = keras.Sequential(
        [
            keras.Input(shape=(n_timesteps,n_features)),
            layers.Conv1D(filters=100, kernel_size=50, activation='relu'),
            layers.Conv1D(filters=1, kernel_size=50, activation='relu'),
            layers.Dropout(0.1),
            layers.MaxPooling1D(pool_size=2),
            layers.Flatten(),
            layers.Dense(100, activation='relu'),
            layers.Dense(n_outputs, activation='softmax')
        ]
    )
    #model.summary()
    return model

In [12]:
print(x_train.shape, y_train.shape)
print(x_validate.shape, y_validate.shape)
print(x_test.shape, y_test.shape)

(16747, 187, 1) (16747, 5)
(5583, 187, 1) (5583, 5)
(5586, 187, 1) (5586, 5)


In [13]:
x_train_shuffled, y_train_shuffled = shuffle(x_train, y_train)
model = cnn_model_1(x_train, y_train)
BATCH_SIZE = 32     # Typical values are 8, 16 or 32
NUM_EPOCHS = 30     # How many epochs to train the deep learning model

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
print(model.summary())
start = time.time()
history = model.fit(
        x_train,y_train,
        steps_per_epoch=x_train.shape[0]//BATCH_SIZE,
        epochs=NUM_EPOCHS,
        validation_data=(x_validate,y_validate),
        validation_steps=x_validate.shape[0]//BATCH_SIZE)

print("Final Validation Accuracy: %0.3f" % history.history['val_accuracy'][-1])
end = time.time()
predictions = model.predict(x_test, verbose=0,batch_size=32)

#must use values not one-hot encoding, use argmax to convert
y_pred = np.argmax(predictions, axis=-1) # axis=-1 means last axis
y_test_act = np.argmax(y_test, axis=-1) # undo one-hot encoding

# Print print prediction accuracy
print('Prediction accuracy: {0:.3f}'.format(accuracy_score(y_test_act, y_pred)))
#acc_list_original.append(accuracy_score(y_test_act, y_pred))
print(end)
print(start)
print(end-start)
del model
del history

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_6 (Conv1D)               │ (None, 138, 100)       │         5,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_7 (Conv1D)               │ (None, 89, 1)          │         5,001 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 89, 1)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_3 (MaxPooling1D)  │ (None, 44, 1)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 44)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 100)            │         4,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 5)              │           505 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 15,106 (59.01 KB)

 Trainable params: 15,106 (59.01 KB)

 Non-trainable params: 0 (0.00 B)

...

503/523 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9362 - loss: 0.1800

2024-04-26 03:17:03.639525: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


523/523 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - accuracy: 0.9363 - loss: 0.1799 - val_accuracy: 0.9226 - val_loss: 0.2436


2024-04-26 03:17:04.043315: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


Final Validation Accuracy: 0.923
Prediction accuracy: 0.910
1714101424.0761561
1714101255.9262364
168.14991974830627


In [5]:
gpus = tf.config.experimental.list_physical_devices('GPU')
    #print("Available GPUs:", gpus)
    
    # Set TensorFlow to only use the first GPU
if gpus:
    try:
        tf.config.experimental.set_visible_devices(gpus[5], 'GPU')
        tf.config.experimental.set_memory_growth(gpus[5], True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
    except RuntimeError as e:
        print(e)

8 Physical GPUs, 1 Logical GPU


2024-04-06 23:55:35.437373: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22501 MB memory:  -> device: 5, name: NVIDIA RTX A5000, pci bus id: 0000:c8:00.0, compute capability: 8.6
